# High Frequency Trading Algorithm

You have been tasked by the investment firm Renaissance High Frequency Trading (RHFT) to develop an automated trading strategy utilizing a combination of machine learning algorithms and high frequency algorithms. RHFT wants this new algorithm to be based on stock market data of the 30 stocks in the Dow Jones at the minute level and to conduct buys and sells every minute based on 1 min, 5 min, and 10 min Momentum. The CIO asked you to choose the Machine Learning Algorithm best suited for this task and wants you to execute the trades via Alpaca's API.

## Part 1: Prepare the data for training and testing

### Initial Set-Up

In [53]:
# Initial Imports
import os
from pathlib import Path
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
import datetime
import time
from dotenv import load_dotenv




In [54]:
# Load .env enviroment variables
load_dotenv()

True

In [55]:
# Set Alpaca API key and secret
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

In [56]:
print(f"Alpaca Key type: {type(API_KEY)}")
print(f"Alpaca Secret Key type: {type(API_SECRET)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [57]:
# Create the Alpaca API object, specifying use of the paper trading account:
api = tradeapi.REST(API_KEY, API_SECRET, ALPACA_API_BASE_URL, api_version='v2')

### Data Generation



#### 1. Create a ticker list, beginning and end dates, and timeframe interval.


In [58]:
# Define a list of tickers
ticker_list = ["TSLA", "MSFT", "AMZN", "FB", "AAPL", "NVDA", "GOOG", "CRM"]

# declare begin and end date strings
beg_date = '2022-05-27'
end_date = '2022-05-27'
# we convert begin and end date to formats that the ALPACA API requires
start =  pd.Timestamp(f'{beg_date} 09:30:00-0400', tz='America/New_York').replace(hour=9, minute=30, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
end   =  pd.Timestamp(f'{end_date} 16:00:00-0400', tz='America/New_York').replace(hour=16, minute=0, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
# We set the time frequency at which we want to pull prices
timeframe='1Min'


#### 2. Ping the Alpaca API for the data and store it in a DataFrame called `prices` by using the `get_barset` function combined with the `df` method from the Alpaca Trade SDK.

In [59]:
# Pull prices from the ALPACA API
prices = api.get_bars(ticker_list, timeframe, limit=1000, start=start, end=end).df


In [60]:
prices.head()

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2022-05-27 13:30:00+00:00,145.39,145.740,145.260,145.540,1718145,16997,145.401780,AAPL
2022-05-27 13:31:00+00:00,145.56,146.130,145.560,146.000,656681,5929,145.872124,AAPL
2022-05-27 13:32:00+00:00,146.00,146.230,145.930,146.180,411564,3584,146.068032,AAPL
2022-05-27 13:33:00+00:00,146.18,146.205,146.020,146.045,374322,3481,146.121218,AAPL
2022-05-27 13:34:00+00:00,146.04,146.110,145.751,145.790,505251,3876,145.933648,AAPL


#### 3. Store only the close prices from the `prices` DataFrame in a new DataFrame called `df_closing_prices`, then view the head and tail to confirm the following:
* First price for each stock on the open at 9:30 Eastern Time.
* Last price for the day on the close at 3:59 pm Eastern Time.

In [61]:
# Create an empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices for each one of the tickers and store in a column in df_closing_prices amed after that ticker
for ticker in ticker_list:
    df_closing_prices[ticker] = prices.loc[prices['symbol'] == ticker]['close']



In [62]:
# Preview first five rows
df_closing_prices.head(5)

,TSLA,MSFT,AMZN,FB,AAPL,NVDA,GOOG,CRM
timestamp,,,,,,,,
2022-05-27 13:30:00+00:00,NaN,NaN,2271.970,NaN,145.540,NaN,NaN,161.0025
2022-05-27 13:31:00+00:00,NaN,NaN,2281.460,NaN,146.000,NaN,NaN,160.6900
2022-05-27 13:32:00+00:00,NaN,NaN,2280.750,NaN,146.180,NaN,NaN,160.9900
2022-05-27 13:33:00+00:00,NaN,NaN,2271.010,NaN,146.045,NaN,NaN,161.1150
2022-05-27 13:34:00+00:00,NaN,NaN,2264.365,NaN,145.790,NaN,NaN,161.0900


In [63]:
# Preview last five rows
df_closing_prices.tail(5)

,TSLA,MSFT,AMZN,FB,AAPL,NVDA,GOOG,CRM
timestamp,,,,,,,,
2022-05-27 19:56:00+00:00,NaN,NaN,2299.99,NaN,149.195,NaN,NaN,NaN
2022-05-27 19:57:00+00:00,NaN,NaN,2299.46,NaN,149.415,NaN,NaN,NaN
2022-05-27 19:58:00+00:00,NaN,NaN,2299.88,NaN,149.510,NaN,NaN,NaN
2022-05-27 19:59:00+00:00,NaN,NaN,2303.38,NaN,149.650,NaN,NaN,NaN
2022-05-27 20:00:00+00:00,NaN,NaN,2301.99,NaN,149.640,NaN,NaN,NaN


In [64]:
# Number of rows
df_closing_prices.shape

(390, 8)

#### 4. When viewing the head and tail, you'll notice several `NaN` values.
* Alpaca reports `NaN` for minutes without any trades occuring as missing.
* These values must be removed, we use Panda's `ffill()` function to "forward fill", or replace, those prices with the previous values (since the price has not changed).


In [ ]:
# Use Pandas' forward fill function to fill missing values (be sure to set inplace=True)
# YOUR CODE HERE

### Computing Returns

#### 1. Compute the percentage change values for 1 minute as follows:
* Create a variable called `forecast` to hold the forecast, in this case `1` for 1 minute.
* Use the `pct_change` function, passing in the `forecast`, on the `df_closing_prices` DataFrame, storeing the newly generated DataFrame in a variable called `returns`.
* Convert the `returns` DataFrame to show forward returns by passing `-(forecast)` into the `shift function.`

In [ ]:
# Define a variable to set prediction period
# YOUR CODE HERE
# Compute the pct_change for 1 min 
# YOUR CODE HERE
# Shift the returns to convert them to forward returns
# YOUR CODE HERE
# Preview the DataFrame
# YOUR CODE HERE

##### Note: 
> You can verify these returns are computed correctly by analyzing the first observation for Facebook:
> * 9:30 am for 0.000632.
 
> How is that number computed? 
 
> * The price of Facebook at 9:30 is 269.00
> * The price of Facebook at 9:31 is 269.17

> Which gives you:

> * (269.17 - 	269.00)/ 269.90 = 0.000632
 

#### 2. Convert the DataFrame into long form for merging later using `unstack` and `reset_index`.

In [ ]:
# Use unstack() to bring the data in long format and save the output as as dataframe
# YOUR CODE HERE
# Rename the column to make it easer to identify it:
name = f'F_{forecast}_m_returns'
returns.rename(columns={0: name}, inplace = True)
# Reset the index of the dataframe for merging later (be sure to set inplace=True)
# YOUR CODE HERE

In [ ]:
# Preview the first five rows
# YOUR CODE HERE

In [ ]:
# Preview the last five rows
# YOUR CODE HERE

#### 3. Compute the 1, 5, 10 minute momentums that will be used to predict the forward returns, then merge them with the forward returns as follows:
* Create the list of moments: `list_of_momentums = [1,5,10]`.
* Write a for-loop to loop through the `list_of_momentums`, applying them to `pct_change` with the `df_closing_price` with each iteration.
* With each loop, the data temporary DataFrame, `returns_temp` will need to be prepped with `unstack` and `reset_index`, then added as a new column to the original `returns` DataFrame from the prior step.
* Complete this step by dropping the null values from `returns` and creating a multi-index based on date and ticker.

In [ ]:
# Create list of momentums that we want to predict
list_of_momentums = [1,5,10]
for i in list_of_momentums:   
    # Compute percentage change for each one of the momentums in the momentum list
    # YOUR CODE HERE
    # Unstack the returns and save the output as as dataframe called returns_temp 
    # YOUR CODE HERE
    # Rename the column to make it easer to identify it:
    # YOUR CODE HERE
    # Reset the index so we can merge based on index
    # YOUR CODE HERE
    # Merge returns_temp  with the original returns 
    returns = pd.merge(returns,returns_temp,left_on=['level_0', 'time'],right_on=['level_0', 'time'], how='left', suffixes=('_original', 'right'))

In [ ]:
returns.head(11)

In [ ]:
# Use dropna() to get rid of those missing observations.
# YOUR CODE HERE
# Create a multi index based on level_0 and time
# YOUR CODE HERE

## Part 2: Train and Compare Multiple Machine Learning Algorithms

 In this section, you'll train each of the requested algorithms and compare performance. Be sure to use the same parameters and training steps for each model. This is necessary to compare each model accurately.

### Preprocessing Data

#### 1. Generate your feature data (`X`) and target data (`y`):
* Create a dataframe `X` that contains all the columns from the returns dataframe that will be used to predict `F_1_m_returns`.
* Create a variable, called `y`, that is equal 1 if `F_1_m_returns` is larger than 0. This will be our target variable.

In [ ]:
# Load the dataset returns.csv and set the index to level_0 and time
# YOUR CODE HERE

In [ ]:
# Create a separate dataframe for features and define the target variable as a binary target
# YOUR CODE HERE

# Create the target variable
# YOUR CODE HERE

##### Note:
> Notice that we don't use shuffle when splitting the dataset into a training and testing dataset. 

> We want to keep the original ordering of the data, so we don't end up using observations in the future to predict past observations,

> This is a critical mistake known as look ahead bias.

#### 2. Use the train_test_split library to split the dataset into a training and testing dataset, with 70% used for testing
* Set the shuffle parameter to False, so that you use the first 70% for training to prvent look ahead bias.
* Make sure you have these 4 variables: `X_train`, `X_test`, `y_train`, `y_test`. 

In [ ]:
# Import train_test_split 
from sklearn.model_selection import train_test_split

# Split the dataset without shuffling
# YOUR CODE HERE

#### 3. Use the `Counter` function to test the distribution of the data. 
* The result of `Counter({1: 668, 0: 1194})` reveals the data is indeed unbalanced.

In [ ]:
# Import the Counter function from the collections library
from collections import Counter

# Use Counter to count the number 1s and 0 in y_train
# YOUR CODE HERE

#### 4. Balance the dataset with the Oversampler libary, setting `random state= 1`.

In [ ]:
# Import RandomOverSampler from the imblearn library
from imblearn.over_sampling import RandomOverSampler

# Use RandomOverSampler to resample the datase using random_state=1
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#### 5. Test the distribution once again with `Counter`. The new result of `Counter({1: 1194, 0: 1194})` shows the data is now balanced.

In [ ]:
# Use Counter again to verify imbalance removed
# YOUR CODE HERE

# Machine Learning

#### 1. The first cells in this section provide an example of how to fit and train your model using the `LogisticRegression` model from sklearn:
* Import select model.
* Instantiate model object.
* Fit the model to the resampled data - `X_resampled` and `y_resampled`.
* Predict the model using `X_test`.
* Print the classification report.

In [ ]:
# Import classification_report from sklearn
from sklearn.metrics import classification_report

In [ ]:
# Import LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression

# Create a LogisticRegression model and train it on the X_resampled data we created before
log_model = LogisticRegression()
log_model.fit(X_resampled, y_resampled)  

# Use the model you trained to predict using X_test
y_pred = log_model.predict(X_test)   

# Print out a classification report toevaluate performance
print(classification_report(y_test, y_pred, digits=4))

#### 2. Use the same approach as above to train and test the following ML Algorithms:
* [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)
* [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)
* [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

#### RandomForestClassifier

In [ ]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Create a RandomForestClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

# Use the model you trained to predict using X_test
# YOUR CODE HERE  

# Print out a classification report to evaluate performance
# YOUR CODE HERE

#### GradientBoostingClassifier

In [ ]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import GradientBoostingClassifier

# Create a GradientBoostingClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

# Use the model you trained to predict using X_test
# YOUR CODE HERE     

# Print out a classification report to evaluate performance
# YOUR CODE HERE

#### AdaBoostClassifier

In [ ]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import AdaBoostClassifier

# Create a AdaBoostClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

# Use the model you trained to predict using X_test
# YOUR CODE HERE

# Print out a classification report to evaluate performance
# YOUR CODE HERE

#### XGBClassifier

In [ ]:
# Import RandomForestClassifier from sklearn
from xgboost import XGBClassifier

# Create a XGBClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

# Use the model you trained to predict using X_test
# YOUR CODE HERE

# Print out a classification report to evaluate performance
# YOUR CODE HERE

### Evaluate the performance of each model


#### 1. Using the classification report for each model, choose the model with the highest precision for use in your algo-trading program.
#### 2. Save the selected model with the `joblib` libary to avoid retraining every time you wish to use it.

In [ ]:
# Import the joblib library 
import joblib

# Use the library to save the model that you want to use for trading
joblib.dump(log_model, 'log_model.pkl')

## Part 3: Implement the strongest model using Apaca API

### Develop the Algorithm


#### 1. Use the provided code to ping the Alpaca API and create the DataFrame needed to feed data into the model.
   * This code will also store the correct feature data in `X` for later use.

In [ ]:
# Create the list of tickers

ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
# Define Dates

beg_date = '2021-01-06'
end_date = '2021-01-06'

# Convert the date in a format the Alpaca API reqires
start =  pd.Timestamp(f'{beg_date} 09:30:00-0400', tz='America/New_York').replace(hour=9, minute=30, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
end   =  pd.Timestamp(f'{end_date} 16:00:00-0400', tz='America/New_York').replace(hour=15, minute=0, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
timeframe='1Min'

# Use iloc to get the last 10 mins every time we pull new data
prices = api.get_barset(ticker_list, "minute", start=start, end=end).df.iloc[-11:]
prices.ffill(inplace=True)   

# Create an empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices of our tickers
df_closing_prices["FB"] = prices["FB"]["close"]
df_closing_prices["AMZN"] = prices["AMZN"]["close"]
df_closing_prices["AAPL"] = prices["AAPL"]["close"]
df_closing_prices["NFLX"] = prices["NFLX"]["close"]
df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
df_closing_prices['MSFT'] = prices['MSFT']["close"]
df_closing_prices['TSLA'] = prices['TSLA']["close"]

print(df_closing_prices.head(20))

In [ ]:
# Create list of momentums
list_of_momentums = [1,5,10]

for i in list_of_momentums:  
    # Compute percentage change for each one of the momentums in the momentum list
    returns_temp = df_closing_prices.pct_change(i)
    # Unstack the returns 
    returns_temp = pd.DataFrame(returns_temp.unstack())
    name = f'{i}_m_returns'
    returns_temp.rename(columns={0: name}, inplace = True)
    # Reset the index so we can merge based on index
    returns_temp.reset_index(inplace = True)
    # Merge newly computed returns with previously created returns
    if i ==1:
        returns = returns_temp
    else:
        returns = pd.merge(returns,returns_temp,left_on=['level_0', 'time'],right_on=['level_0', 'time'], how='left', suffixes=('_original', 'right'))

# Drop nulls and set index
returns.dropna(axis=0, how='any', inplace=True)
returns.set_index(['level_0', 'time'], inplace=True)

# Generate feature data and preview first 10 rows.
X = returns
X.head(10)

#### 2. Using `joblib`, load the chosen model.

In [ ]:
# Load the previously trained and saved model using joblib
# YOUR CODE HERE

#### 3. Use the model file to make predicttions:
* Use `predict` on `X` and save this as `y_pred`.
* Convert `y_pred` to a DataFrame, setting the index to the index of `X`.
* Rename the column 0 to 'buy', be sure to set `inplace =True`.

In [ ]:
# Use the model file to predict on X
# YOUR CODE HERE

# Convert y_pred to a dataframe, set the index to the index of X
# YOUR CODE HERE

# Rename the column 0 to 'buy', be sure to set inplace =True
# YOUR CODE HERE

#### 4. Filter the stocks where 'buy' is equal to 1, saving the filter as `y_pred`.

In [ ]:
# Filter the stocks where 'buy' is equal to 1
# YOUR CODE HERE

#### 5. Using the `y_pred` filter, create a dictionary called `buy_dict` and assign 'n' to each Ticker (key value) as a placeholder.

In [ ]:
# Create dictionary from y_pred and assign a 'n' to each of them for now as a placeholder.
buy_dict = dict.fromkeys(y_pred.index.get_level_values(0), 'n')
buy_dict

#### 6. Obtain the total available equity in your account from the Alpaca API and store in a variable called `total_capital`. You will split the capital equally between all selected stocks per the CIO's request.

In [ ]:
# Pull the total available equity in our account from the  Alpaca API
# YOUR CODE HERE

In [ ]:
# Compute capital per stock, divide equity in account by number of stocks
# Use Alpaca API to pull the equity in the account
if len(buy_dict) > 0:
    capital_per_stock = float(total_capital)/ len(buy_dict)
else:
    capital_per_stock = 0
print(f'Capital per stock: {capital_per_stock}')

#### 7. Use a for-loop to iterate through `buy_dict` to determine the number stocks you need to buy for each ticker.

In [ ]:
# Use for loop to iterate through dictionary of buys 
# Determine the number stocks we need to buy for each ticker
for ticker in buy_dict:
    try:
        buy_dict[ticker] = int(capital_per_stock /int(prices[ticker].iloc[-1]['close']))
    except:
        pass

print(buy_dict)

#### 8. Cancel all previous orders in the Alpaca API (so you don't buy more than intended) and sell all currently held stocks to close all positions.

In [ ]:
# Cancel all previous orders in the Alpaca API
# YOUR CODE HERE

# Sell all currently held stocks to close all positions
# YOUR CODE HERE

#### 9. Iterate through `buy_dict` and send a buy order for each ticker with their corresponding number of shares.

In [ ]:
# Iterate through the longlist object and send a buy order for each ticker with a corresponding number of shares:
# YOUR CODE HERE

### Automate the algorithm

#### 1. Make a function called `trade()` that incorporates all of the steps above.

In [ ]:
# Add all of the steps conducted above into the function trade
def trade():

    ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
    # Notice that we remove the start and end variables since we want the latest prices.
    timeframe='1Min'
    # Use iloc to get the last 10 mins every time we pull new data
    prices = api.get_barset(ticker_list, "minute").df.iloc[-11:]
    prices.ffill(inplace=True)   

    # Create and empty DataFrame for closing prices
    df_closing_prices = pd.DataFrame()

    # Fetch the closing prices of our tickers
    df_closing_prices["FB"] = prices["FB"]["close"]
    df_closing_prices["AMZN"] = prices["AMZN"]["close"]
    df_closing_prices["AAPL"] = prices["AAPL"]["close"]
    df_closing_prices["NFLX"] = prices["NFLX"]["close"]
    df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
    df_closing_prices['MSFT'] = prices['MSFT']["close"]
    df_closing_prices['TSLA'] = prices['TSLA']["close"]
    print(df_closing_prices.head())
    
    # Loop through momentums to build new DataFrame
    list_of_momentums = [1,5,10]
    for i in list_of_momentums:   
        returns_temp = df_closing_prices.pct_change(i)
        returns_temp = pd.DataFrame(returns_temp.unstack())
        name = f'{i}_m_returns'
        returns_temp.rename(columns={0: name}, inplace = True)
        returns_temp.reset_index(inplace = True)
        if i ==1:
            returns = returns_temp
        else:
            returns = pd.merge(returns,returns_temp,left_on=['level_0', 'time'],right_on=['level_0', 'time'], how='left', suffixes=('_original', 'right'))

    # Drop nulls and set index            
    returns.dropna(axis=0, how='any', inplace=True)
    returns.set_index(['level_0', 'time'], inplace=True)

    # Preprocess data for model
    # YOUR CODE HERE

    # Create the `buy_dict` object
    # YOUR CODE HERE
    
    # Split capital between stocks and determine buy or sell
    # YOUR CODE HERE

    
    # Cancel pending orders and close positions
    # YOUR CODE HERE
   
    
    # Submit orders
    # YOUR CODE HERE


#### 2. Import Python's schedule module.

In [ ]:
# Import Python's schedule module 
# YOUR CODE HERE

#### 3. Use the "schedule" module to automate the algorithm:
* Clear the schedule with `.clear()`.
* Define a schedule to run the trade function every minute at 5 seconds past the minute mark (e.g. `10:31:05`).
* Use the Alpaca API to check whether the market is open.
* Use run_pending() function inside schedule to execute the schedule you defined while the market is open

In [ ]:
# Clear the schedule
# YOUR CODE HERE

# Define a schedule to run the trade function every minute at 5 seconds past the minute mark (e.g. 10:31:05)
# YOUR CODE HERE

# Use the Alpaca API to check whether the market is open
# YOUR CODE HERE

# Use run_pending() function inside schedule to execute the schedule you defined as long as the market is open
# YOUR CODE HERE